In [1]:
%run -m ipy_startup
%run -m ipy_logging
%run -m ipy_seaborn
%matplotlib inline
from mgds.data_aggregation import api
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import data_type as dtyp
from mgds.data_aggregation import entity
from mgds.data_modeling import data_broker as dbm

In [2]:
entity_mappings={
    entity.CELL_LINE: api.get_entity_mapping(entity.CELL_LINE),
    entity.PRIMARY_SITE: api.get_entity_mapping(entity.PRIMARY_SITE)
}

In [24]:
df = api.get_raw_genomic_data(src.NCIDREAM_v1, dtyp.DRUG_SENSITIVITY, mappings=entity_mappings)
df['INDICATOR'] = 1

In [25]:
df.head()

,CELL_LINE_ID,DRUG_NAME,LN_GI50,LN_GI50_STD,REPLICATE_COUNT,TARGET_PATHWAY,CELL_LINE_ID:MGDS,PRIMARY_SITE:MGDS,PRIMARY_SITE:SOURCE,DRUG_NAME:MGDS,DRUG_NAME:SOURCE,INDICATOR
0,184A1,B581,1.429324,0.000000,1.0,FTPase,184A1,BREAST,BREAST,B581,B581,1
1,184A1,Baicalein,1.329005,0.000000,1.0,CYP2C9,184A1,BREAST,BREAST,BAICALEIN,Baicalein,1
2,184A1,Cetuximab,1.976354,1.487831,2.0,EGFR,184A1,BREAST,BREAST,CETUXIMAB,Cetuximab,1
3,184A1,Disulfiram,1.589160,0.000000,1.0,ALDH2,184A1,BREAST,BREAST,DISULFIRAM,Disulfiram,1
4,184A1,Everolimus,1.818879,0.000000,1.0,mTOR,184A1,BREAST,BREAST,EVEROLIMUS,Everolimus,1


In [28]:
df.groupby(['CELL_LINE_ID:MGDS', 'GENE_ID:HGNC', 'VALUE']).size().value_counts()
#df.groupby(['CELL_LINE_ID:MGDS', 'GENE_ID:HGNC', 'INDICATOR']).size().value_counts()

1    17000944
dtype: int64

In [29]:
df.groupby(['CELL_LINE_ID:MGDS', 'GENE_ID:HGNC']).size().value_counts()

1    17000944
dtype: int64

In [21]:
import imp
imp.reload(api)

<module 'mgds.data_aggregation.api' from '/Users/eczech/repos/mgds/python/src/mgds/data_aggregation/api.py'>

In [22]:
datasets = api.get_genomic_data_availability()
datasets = [v for v in datasets if v[1] == dtyp.DRUG_SENSITIVITY]
datasets

[('gdsc_v2', 'drug-sensitivity'),
 ('ccle_v1', 'drug-sensitivity'),
 ('nci60_v2', 'drug-sensitivity'),
 ('ncidream_v1', 'drug-sensitivity')]

In [23]:
from py_utils.pandas_utils import one_value

def get_feature_datasets(dataset_info):
    d = []
    for (source, data_type) in dataset_info:
        logger.info('Unpacking source "{}", data set "{}"'.format(source, data_type))

        c_val = 'INDICATOR' if data_type in [dtyp.GENE_EXOME_SEQ] else 'VALUE'
        agg_func = 'mean' if data_type in [dtyp.GENE_EXOME_SEQ] else one_value

        # Leaving this out due to lack of clarity on encoding (what is "-1,-1,-,-"?)
        if source == src.GDSC_v2 and data_type in [dtyp.GENE_COPY_NUMBER]:
            continue

        # These were both verified to show that the values associated with all duplicate MGDS cell lines
        # and genes are identical (so the mean will be used, but there is no variance)
        if source == src.CCLE_v1 and data_type in [dtyp.GENE_COPY_NUMBER, dtyp.GENE_EXPRESSION]:
            agg_func = 'mean'

    #     if source == src.GDSC_v2 or source == src.NCI60_v2 or source == src.CCLE_v1:
    #         continue


        df = api.get_raw_genomic_data(source, data_type, mappings=entity_mappings)

        if c_val == 'VALUE' and 'VALUE' not in df:
            c_val = 'VALUE_MEAN'

        df['INDICATOR'] = 1


        df = df.pivot_table(
            index=['CELL_LINE_ID:MGDS', 'PRIMARY_SITE:MGDS'],
            columns='GENE_ID:HGNC',
            values=c_val,
            aggfunc=agg_func
        )
        df.columns = pd.MultiIndex.from_tuples([(source, data_type, c) for c in df])
        d.append(df)
    return pd.concat(d, axis=1)

datasets = api.get_genomic_data_availability()
datasets = [v for v in datasets if v[1] == dtyp.DRUG_SENSITIVITY]

d = get_feature_datasets(datasets)
#d = db.cache_prep_operation(lambda: get_feature_datasets(datasets), 'raw-features', overwrite=False)

2017-01-20 15:08:27,491:INFO:root: Unpacking source "gdsc_v2", data set "drug-sensitivity"


KeyError: 'GENE_ID:HGNC'

In [4]:
d.head()

ccle_v1                   \
                                         gene-copy-number                    
                                                     A1BG A1BG-AS1    A1CF   
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                          
1321N1            CENTRAL_NERVOUS_SYSTEM          -0.1544  -0.1544 -0.0985   
143B              BONE                            -0.2120  -0.2120  0.1580   
184A1             BREAST                              NaN      NaN     NaN   
184B5             BREAST                              NaN      NaN     NaN   
201T              LUNG                                NaN      NaN     NaN   

                                                                          \
                                                                           
                                             A2M   A2ML1  A4GALT   A4GNT   
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                        
1321N1            CENTRAL_NERVOUS_SYSTEM -0.1819 -0.1819 -0.1732  0.2575   
143B              BONE                   -0.1753 -0.1753 -0.2536  0.2060   
184A1             BREAST                     NaN     NaN     NaN     NaN   
184B5             BREAST                     NaN     NaN     NaN     NaN   
201T              LUNG                       NaN     NaN     NaN     NaN   

                                                                   ...     \
                                                                   ...      
                                            AAAS    AACS  AACSP1   ...      
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                ...      
1321N1            CENTRAL_NERVOUS_SYSTEM -0.0404 -0.1284  0.1682   ...      
143B              BONE                   -0.1445 -0.2757  0.2150   ...      
184A1             BREAST                     NaN     NaN     NaN   ...      
184B5             BREAST                     NaN     NaN     NaN   ...      
201T              LUNG                       NaN     NaN     NaN   ...      

                                          ncidream_v1                    \
                                         gene-rna-seq                     
                                               snoR38 snoU2-30 snoU2_19   
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                       
1321N1            CENTRAL_NERVOUS_SYSTEM          NaN      NaN      NaN   
143B              BONE                            NaN      NaN      NaN   
184A1             BREAST                          0.0      0.0      0.0   
184B5             BREAST                          0.0      0.0      0.0   
201T              LUNG                            NaN      NaN      NaN   

                                                                             \
                                                                              
                                          snoU6-53    snoU83B snoZ30 snoZ39   
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                           
1321N1            CENTRAL_NERVOUS_SYSTEM       NaN        NaN    NaN    NaN   
143B              BONE                         NaN        NaN    NaN    NaN   
184A1             BREAST                  0.000000  12.001058    0.0    0.0   
184B5             BREAST                  5.493734  16.925710    0.0    0.0   
201T              LUNG                         NaN        NaN    NaN    NaN   

                                                                       
                                                                       
                                         snoZ40 snosnR60_Z15 snosnR66  
CELL_LINE_ID:MGDS PRIMARY_SITE:MGDS                                    
1321N1            CENTRAL_NERVOUS_SYSTEM    NaN          NaN      NaN  
143B              BONE                      NaN          NaN      NaN  
184A1             BREAST                    0.0    48.899581      0.0  
184B5             BREAST                    0.0    53.819947      0.0  
201T

In [5]:
sites = d.index.get_level_values('PRIMARY_SITE:MGDS')
sites.value_counts()

HAEMATOPOIETIC_AND_LYMPHOID_TISSUE    243
LUNG                                  229
BREAST                                 86
SKIN                                   86
CENTRAL_NERVOUS_SYSTEM                 77
LARGE_INTESTINE                        68
OVARY                                  66
BONE                                   60
UPPER_AERODIGESTIVE_TRACT              59
KIDNEY                                 53
PANCREAS                               49
STOMACH                                48
AUTONOMIC_GANGLIA                      39
OESOPHAGUS                             37
URINARY_TRACT                          30
ENDOMETRIUM                            29
LIVER                                  28
SOFT_TISSUE                            28
PLEURA                                 26
THYROID                                18
CERVIX                                 14
BILIARY_TRACT                          12
PROSTATE                               10
UNKNOWN                           

In [6]:
# from sklearn.model_selection import LeaveOneGroupOut, StratifiedKFold, KFold
# #cv = LeaveOneGroupOut(['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c'])
# cv = KFold()
# cv.split()

In [7]:
from sklearn.model_selection import KFold
from py_utils import collection_utils
import collections

def _validate_fold(folds, i, j, k):
    #     print(len(folds[i]['test']))
    #     print(len(folds[i]['inner'][j]['train']))
    #     print(len(folds[i]['inner'][j]['test']))
    assert len(folds[i]['test']) == \
        len(folds[i]['inner'][j]['train']) + len(folds[i]['inner'][j]['test'])
    assert len(folds[i]['inner'][j]['train']) == \
        len(folds[i]['inner'][j]['inner'][k]['train']) + len(folds[i]['inner'][j]['inner'][k]['test'])

    
def get_cv_folds(d, sites=None, n_split1=10, n_split2=3, random_state=None):
    idx = pd.Series(np.arange(len(d)))
    all_sites = d.index.get_level_values('PRIMARY_SITE:MGDS')
    
    if sites is None:
        sites = sorted(np.unique(all_sites))

    folds = collections.OrderedDict()
    for i, site in enumerate(sites):
        train_0 = idx.loc[all_sites != site].values
        test_0 = np.setdiff1d(idx, train_0)

        folds[i] = {
            'train': idx.iloc[train_0], 
            'test': idx.iloc[test_0], 
            'site': site, 
            'inner': collections.OrderedDict()
        }
        cv_1 = KFold(n_splits=n_split1, random_state=random_state, shuffle=True)
        idx_1 = folds[i]['test']
        
        for j, (train_1, test_1) in enumerate(cv_1.split(idx_1)):
            folds[i]['inner'][j] = {
                'train': idx_1.iloc[train_1], 
                'test': idx_1.iloc[test_1], 
                'inner': collections.OrderedDict()
            }
            cv_2 = KFold(n_splits=n_split2, random_state=random_state, shuffle=True)
            idx_2 = folds[i]['inner'][j]['train']
            
            for k, (train_2, test_2) in enumerate(cv_2.split(idx_2)):
                folds[i]['inner'][j]['inner'][k] = {
                    'train': idx_2.iloc[train_2], 
                    'test': idx_2.iloc[test_2]
                }
                _validate_fold(folds, i, j, k)
    return folds

In [8]:
cv_sites = ['BREAST', 'LUNG', 'SKIN']
cv = get_cv_folds(d, sites=cv_sites)

TODO:

- Train on Pan-Site to determine best estimators
- Train TCGA RPP

In [9]:
x = {}
def test():
    return (1, 2)
_, x['x'] = test()


In [10]:
x

{'x': 2}

In [14]:
d.columns.nlevels

3

In [15]:
tuple(np.repeat('fold', 3))

('fold', 'fold', 'fold')

In [66]:
from sklearn import base

def xy(d, cx, cy):
        assert len(np.intersect1d(cx, cy)) == 0
        assert len(cx) > 0
        assert len(cy) > 0
        return d[cx], d[cy] 
        
def train_models(models, d_train, d_test=None, prefit_models=None, include_predictions=False):
    estimators = {}
    predictions = []

    responses = models['response_selector'](d_train)
    Y_test = None

    # Train each model individually
    for est_name in models['estimators']:
        est_def = models[est_name]

        # Determine features and responses for this model
        features = est_def['feature_selector'](d_train)

        X_train, Y_train = xy(d_train, features, responses)

        X_test, Y_test = None, None
        if d_test is not None:
            X_test, Y_test = xy(d_test, features, responses)

        prefit_est = None
        if prefit_models is not None and est_name in prefit_models:
            prefit_est = base.clone(prefit_models[est_name])

        est, Y_pred = est_def['train'](X_train, Y_train, X_test, prefit_est)
        Y_pred.columns = pd.MultiIndex.from_tuples([(*c, est_name) for c in Y_pred])

        if Y_pred is not None and est is not None:
            predictions.append(Y_pred)
            estimators[est_name] = est

    # Concatenate predictions side by side noting that the predictions from the individual models
    # do not necessarily need to be for the same things
    Y_pred = pd.concat(predictions, axis=1) if len(predictions) > 0 else None

    if include_predictions:
        if Y_test is not None:
            Y_test.columns = pd.MultiIndex.from_tuples([(*c, 'Actual') for c in Y_test])
        estimators, Y_pred, Y_test
    else:
        estimators
        

    
def add_fold_id(d, fold_id):
    fold_col = tuple(np.repeat('Fold'), d.columns.nlevels)
    return d.assign(fold_col=fold)

def partition(d, i_train, i_test, fold):
    d_train = d.iloc[i_train].copy()
    d_test = d.iloc[i_test].copy()
    return d_train, d_test


def predict(d_train, d_test, per_site_estimators, pan_site_estimators, prefit_models):
    # Create inner predictions from per-site models, which generally means
    # they will only be trained using the training data available in "d_train"
    per_site_models, Y_pred_1, Y_true_1  = train_models(
        per_site_estimators, d_train, d_test,
        include_predictions=True
    )

    # Create predictions from pan-site models, with training data generally from:
    # 1. Data not for this particular primary site
    # 2. The current inner fold (ie "d_train")
    # 3. Any combination of the above
    # 4. None of the above -- these models could have been trained on something external
    #    and only the predictions produced here are used as part of the meta model input
    pan_site_models, Y_pred_2, Y_true_2 = train_models(
        pan_site_estimators, d_train, d_test,
        prefit_models=prefit_models,
        include_predictions=True
    )
    assert Y_true_1.equals(Y_true_2)
    Y_true = Y_true_1

    Y_pred = pd.concat([Y_pred_1, Y_pred_2], axis=1)

    #return per_site_models, pan_site_models, Y_pred, Y_true
    return Y_pred, Y_true

def run_site_cv(d, site, cv, perse, panse, me, pansepre):
    # Run outer CV loop (this loop will be used to determine predictions
    # from the second-level "meta" model)

    prediction_data = []
    fold_results = collections.OrderedDict()
    for i_outer in cv:
        d_train = d.iloc[cv[i_outer]['train']]
        d_test = d.iloc[cv[i_outer]['test']]

        Y_pred_outer, Y_true_outer = predict(d_train, d_test, perse, panse, pansepre)
        Y_feat_outer = pd.concat([Y_pred_outer, Y_true_outer], axis=1)

        cv_inner = cv[i_outer]['inner']
        Y_feat_inner = []
        for i_inner in cv_inner:
            d_train = d.iloc[cv_inner[i_inner]['train']]
            d_test = d.iloc[cv_inner[i_inner]['test']]
            Y_pred, Y_true = predict(d_train, d_test, perse, panse, pansepre)
            Y_feat_inner.append(pd.concat([Y_pred, Y_true], axis=1))
            del Y_pred, Y_true

        # Concatenate predictions from submodels into training set for meta model
        Y_feat_inner = pd.concat(Y_feat_inner)
        assert Y_feat_inner.columns == Y_feat_outer.columns

        meta_models, Y_pred_meta, Y_true_meta = train_models(
            meta_estimators, 
            Y_feat_inner, 
            Y_feat_outer, 
            include_predictions=True
        )

        assert Y_true_meta.equals(Y_true_outer)
        Y_pred = pd.concat([
            Y_pred_outer,
            Y_pred_meta,
            Y_true_meta
        ], axis=1)

        prediction_data.append(add_fold_id(Y_pred, i_outer))
        
    return {
        'prediction_data': pd.concat(prediction_data),
        'fold_results': fold_results
    }


def run_training(d, cv, pan_site_estimators, per_site_estimators, meta_estimators):
        
    panse = pan_site_estimators
    perse = per_site_estimators
    
    results = {}
    
    # Fit pan-site models across entire dataset (all sites)
    pan_site_refit_models = train_models(pan_site_estimators, d)
    
    for i_site in cv:
        d_train = d.iloc[cv[i_site]['train']]
        d_test = d.iloc[cv[i_site]['test']]
        
        # Train pan-site models on data excluding this primary site
        pan_site_prefit_models = train_models(pan_site_estimators, d_train)
        
        # Run "refit", pre-site models on all data available for this site (ie the 
        # "test" set for the site-based CV indexing)
        per_site_refit_models = train_models(per_site_estimators, d_test)
        
        cv_site = cv[i_site]['inner']
        site = cv[i_site]['site']
        site_results = run_site_cv(
            d, site, cv_site, per_site_estimators, pan_site_estimators, 
            meta_estimators, pan_site_prefit_models
        )
        
        meta_refit_models = train_models(meta_estimators, site_results['prediction_data'])
        site_results['models'] = {
            'pan_site': pan_site_refit_models,
            'per_site': per_site_refit_models,
            'meta': meta_refit_models
        }
        
        # Store results for this primary site
        results[site] = site_results
        
    return results
        

90

In [20]:
d[(src.GDSC_v2, dtyp.DRUG_SENSITIVITY)].head()

/Users/eczech/anaconda/envs/mgds/lib/python3.5/site-packages/ipykernel/kernelbase.py:390: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


KeyError: 'drug-sensitivity'

In [ ]:
#response_fields = [c for c in d if c[1] == dtyp.DRUG_SENSITIVITY]


# "train" function
#             if 'est_predict' in clf_def:
#                 assert 'est_factory' not in clf_def, \
#                     'Estimator definition cannot contain custom training function AND estimator factory '\
#                     'function (implies default training procedure)'
#                 est, Yh = clf_def['est_predict'](X_train, Y_train, X_test, Y_test)
#             else:  
#                 est = clf_def['est_factory']().fit(X_train, Y_train)
#                 Yh = pd.DataFrame(est.predict(X_test), index=X_test.index, columns=X_test.columns)
#                 Yh.columns = pd.MultiIndex.from_tuples([(*c, clf_name) for c in X_test])


def pan_site_response_selector(d):
    pass

def pan_site_feature_selector(d):
    pass

def default_trainer(X_train, Y_train, X_test, prefit_est):
    pass

pan_site_estimators = {
    'response_selector': pan_site_response_selector,
    'estimators': {
        'rf_ccle_ge': {
            'feature_selector': [],
            'train': default_trainer
        }   
    }
}

# These can be essentially the same as the pansite_estimators
inner_estimators = {
    'rf_ccle_ge': {
        'feat_select': [],
        'est_factory': lambda: 
    }
}

outer_estimators = {
    
}
